<a href="https://colab.research.google.com/github/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminario/Seminario_(Grupos_Homogeneos)_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Jaime Díaz González <br>
Url: https://github.com/paella55/03MAIR-Algoritmos-de-optimizacion/blob/master/Seminario/Seminario_(Grupos_Homogeneos)_Algoritmos.ipynb<br>
Problema:
> 1. Elección de grupos de población homogéneos 

Descripción del problema: 
Se solicita diseñar un algoritmo para conseguir una agrupación de terneros en grupos de tratamientos que cumplan de la
mejor manera posible las especificaciones de la productora.

(*) La respuesta es obligatoria





                                        

 (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones? ****<br>



**¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.**




Respuesta:<br>
Sin restricciones, tenemos un total de N! soluciones.

Con restricciones:

Teniendo en cuenta que tenemos un total de N terneros que tenemos que dividir en 3 tratamientos diferentes por 3 grupos cada uno:

Dividimos los terneros en 3 tratamientos distintos -> N/3

Como los tratamientos son independientes entre sí, vamos a empezar a contar con un solo tratamiento.

N/3 terneros / 3 grupos  = N/9. Sabiendo que las permutaciones de terneros dentro del mismo grupo no cambian nada, tenemos un total de (N/9)! posibilidades por cada tratamiento.

Ahora sumamos los tratamientos -> 3*(N/9)! en total

**Modelo para el espacio de soluciones<br>**
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta:

Desde el principio he decido utilizar dos librerias muy interesantes para el manejo de estructuras de datos.

He utilizado Panda DataFrames para simular los datos que me da el cliente de sus terneros. Es una tabla de dos columnas (Peso y Sexo) que en cada instancia guardan un valor. Panda tiene una gran cantidad de funciones interesantes para filtrar datos, además de que visualmente queda bastante organizado. Por todas estas razones lo he utilizado.

Para el espacio de soluciones, he decidido utilizar Numpy, que es una libreria muy utilizada en el ámbito matemático para el manejo de vectores y matrices.

Para cada tratamiento tendremos una matriz de 3 filas. Cada fila serán los grupos. Dentro de cada grupo tendremos los individuos. La forma de acceder al grupo 2, al individuo 3 sería: grupos[1][2] porque los índices empiezan por 0.

Numpy, entre otras cosas, me ayuda a calcular la media y la desviación típica de un vector.

### Creación de la estructura de datos

In [104]:
# librería externa para guardar los datos en tablas
import pandas as pd
from pandas import Series, DataFrame

import random
# liberia externa para manejo de vectores
import numpy as np

# Número de terneros
N = 80
# Número de tratamientos
tratamientos = 3
# Número de grupos
grupos = tratamientos*3

# Generamos datos
# Siguiendo la desviación estandar y la media del peso
pesos_terneros = np.random.uniform(low=35.0, high=39.1, size=(N,))
# Ahora hacemos machos y hembras
machos_hembras = []
for i in range(N):
  machos_hembras.append(random.choice('MH'))

def crear_tabla():
  # Me creo las columnas y le añado los valores
  terneros_dict = {"Peso": pesos_terneros, "Sexo": machos_hembras}
  # Creo la tabla que almacena los valores
  terneros_tabla = DataFrame(terneros_dict)
  return terneros_tabla

terneros_tabla = crear_tabla()
# Muestro 5 de entre todos
terneros_tabla.sample(5)

,Peso,Sexo
14,37.005303,M
64,37.820482,M
79,37.408008,H
25,36.190194,M
24,37.195404,M


### Filtrado de datos

In [0]:
# Ahora vamos a organizar los datos por sexo
terneros_machos = terneros_tabla[terneros_tabla['Sexo'] == 'M']
terneros_hembras = terneros_tabla[terneros_tabla['Sexo'] == 'H']

# Reiniciamos sus indices
terneros_machos = terneros_machos.reset_index(drop=True)
terneros_hembras = terneros_hembras.reset_index(drop=True)

# Ahora voy a ver cuantos terneros tengo que descartar, porque los grupos
# tienen que ser iguales en número.
machos_sobrantes = len(terneros_machos.index) % grupos
hembras_sobrantes = len(terneros_hembras.index) % grupos

# Descartamos los mayores y los menores pesos (outliers)
def descartar_terneros(df: DataFrame, num_descartar):
  
  for i in range(num_descartar):
    # En la iteración par, mayores
    if i % 2 == 0:
      indice = df['Peso'].idxmax()
      df = df.drop(df.index[indice])
      df = df.reset_index(drop=True)
    # En la iteración impar, menores
    else:
      indice = df['Peso'].idxmin()
      df = df.drop(df.index[indice])
      df = df.reset_index(drop=True)
  return df

# Descartamos terneros macho y reiniciamos los indices
terneros_machos = descartar_terneros(terneros_machos, machos_sobrantes)
terneros_machos = terneros_machos.reset_index(drop=True)
# Descartamos terneros hembra y reiniciamos los indices
terneros_hembras = descartar_terneros(terneros_hembras, hembras_sobrantes)
terneros_hembras = terneros_hembras.reset_index(drop=True)

### Crear matriz de tratamientos

In [106]:
# Intentamos encontrar los mínimos, para tener grupos iniciales mejores
def encontrar_minimos(df: DataFrame, num_minimos):
  # Creamos un nuevo DataFrame vacio
  df_minimos = pd.DataFrame()
  for i in range(num_minimos):
    # Buscamos el mínimo y se lo añadimos
    indice = df['Peso'].idxmin()
    df_minimos = df_minimos.append(df.iloc[indice])
    # Lo borramos para que no vuelva a aparecer y refrescamos los indices
    df = df.drop(df.index[indice])
    df = df.reset_index(drop=True)
  
  df_minimos = df_minimos.reset_index(drop=True)
  return df_minimos, df

def crear_matriz_tratamientos(tratamientos, grupos, tamanyo_grupos, df: DataFrame):

  # Creo una matriz tridimensional vacia para tener todos los tratamientos, 
  # todos los grupos y todos los individuos
  matriz_grupos = np.zeros(shape=(tratamientos, grupos, tamanyo_grupos))

  for t in range(tratamientos):
    for g in range(3):
      # Busco los mínimos y los añado al tratamiento y grupo
      (terneros_min, df) = encontrar_minimos(df, tamanyo_grupos)

      individuos = matriz_grupos[t][g]
      # Necesito los indices para poner los valores de los individuos
      indices = [i for i in range(individuos.size)]
      np.put(individuos, indices, terneros_min['Peso'])
      # Finalmente pongo los individuos en la matriz
      matriz_grupos[t][g] = individuos
  
  return matriz_grupos

tamanyo_grupos = len(terneros_machos.index)//grupos

matriz_grupos_machos = crear_matriz_tratamientos(tratamientos, 3,
                                                 tamanyo_grupos, terneros_machos)

matriz_grupos_hembras = crear_matriz_tratamientos(tratamientos, 3,
                                                 tamanyo_grupos, terneros_hembras)

print("Tratamiento 1 grupo 3: " + str(matriz_grupos_machos[0][2]))
print("Tratamiento 2 grupo 1: " + str(matriz_grupos_hembras[1][0]))

Tratamiento 1 grupo 3: [36.1325738  36.19019357 36.21055507 36.21184921]
Tratamiento 2 grupo 1: [35.98288195 36.04124141 36.10662781 36.1418612 ]


**Según el modelo para el espacio de soluciones<br>**
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta: 

He decidido que el error de una solución se calcula mediante la siguiente formula:



1.   Sumo las desviaciones de cada grupo...



> desviacion_total = sumatorio(desviacion_grupo_i)



2.   Calculo la media de cada grupo, y resto sus diferencias. Si las medias son más cercanas, el valor de la diferencia será menor. Todo en valor absoluto para tener valores positivos

> diferencia_medias = |sumatorio(media_i - media_j)|

La función objetivo a minimizar será:



> F = desviacion_total + diferencia_medias

Es por tanto un problema de **minimización**.




### Función objetivo

In [0]:
# Vamos ahora a crear un método para calcular la función objetivo
def funcion_objetivo(grupos):
  desviacion_total = 0
  medias = []
  # Recorremos todos los grupos
  for i in range(len(grupos)):
    desviacion_total = desviacion_total + np.std(grupos[i])
    medias.append(np.average(grupos[i])) 
  
  # Calculamos la diferencia entre medias:
  diferencia_medias = 0
  for i in range(len(medias)-1):
    diferencia_medias = diferencia_medias + abs(medias[i]-medias[i+1])
  return desviacion_total, diferencia_medias

def es_mejor_solucion(grupos, error_actual):
  (desv, med) = funcion_objetivo(grupos)
  error_siguiente = desv + med
  if (error_siguiente < error_actual):
    return True, error_siguiente
  else:
    return False, error_actual

### Crear entorno soluciones

In [0]:
# Me sirve para crear una nueva variable por valor
import copy

def crear_entorno_soluciones(grupos_iniciales):
  # Me sirve para crear una nueva variable por valor
  mejor_solucion = copy.deepcopy(grupos_iniciales)
  
  # Calculo el error inicial
  (desv, med) = funcion_objetivo(mejor_solucion)
  error_actual = desv + med

  # Por cada grupo excepto el último porque ya habrá sido utilizado
  for i in range(len(grupos_iniciales)-1):
    # Por cada ternero del grupo
    for j in range(len(grupos_iniciales[i])):
      # Por cada ternero del siguiente grupo
      for k in range(len(grupos_iniciales[i+1])):
        # Vuelvo a la solución inicial
        grupos = copy.deepcopy(grupos_iniciales)
        # Intercambio terneros distintos cada vez
        tmp = grupos[i+1][k]
        grupos[i+1][k] = grupos[i][j]
        grupos[i][j] = tmp
        
        # Compruebo si la solución actual es mejor
        (es_mejor, error_siguiente) = es_mejor_solucion(grupos, error_actual)
        if (es_mejor):
          error_actual = error_siguiente
          mejor_solucion = copy.deepcopy(grupos)
  
  return mejor_solucion, error_actual

### Resolución por fuerza bruta

Diseña un algoritmo para resolver el problema por fuerza bruta

In [111]:
def fuerza_bruta(tratamiento_inicial):
  # Me sirve para crear una nueva variable por valor
  mejor_solucion = copy.deepcopy(tratamiento_inicial)
  
  # Calculo el error inicial
  (desv, med) = funcion_objetivo(mejor_solucion)
  error_actual = desv + med
  
  while(True):
    actual, error_siguiente = crear_entorno_soluciones(mejor_solucion)
    # Esto signfica que no hemos mejorado nada, debemos parar
    if (error_actual == error_siguiente):
      break
    else:
      error_actual = error_siguiente
      mejor_solucion = copy.deepcopy(actual)
      
  return mejor_solucion, error_actual

def imprimir_mejores(matriz):
  # Por cada tratamiento
  for i in range(len(matriz)):
    print("Solución tratamiento: " + str(i))
    (solucion, error) = fuerza_bruta(matriz[i])
    print(solucion)
    print("Error (desviación + media): " + str(error) + "\n")

print("SOLUCIONES HEMBRAS...\n")
imprimir_mejores(matriz_grupos_hembras)
print("SOLUCIONES MACHOS...\n")
imprimir_mejores(matriz_grupos_machos)

SOLUCIONES HEMBRAS...

Solución tratamiento: 0
[[35.15138435 35.29687589 35.30369393 35.37521029]
 [35.69453293 35.71552288 35.7773687  35.86294281]
 [35.88926591 35.93814829 35.95690236 35.97306195]]
Error (desviación + media): 0.8357507391191414

Solución tratamiento: 1
[[35.98288195 36.04124141 36.10662781 36.1418612 ]
 [36.46800851 36.55933876 36.95037648 37.40800754]
 [37.46859008 37.60703508 37.73005242 37.78534212]]
Error (desviación + media): 2.1339912830700807

Solución tratamiento: 2
[[37.81583843 37.88684271 37.90923522 37.91739057]
 [37.98050248 38.14923344 38.26640197 38.49860193]
 [38.53031497 38.67306077 38.83918749 39.01881702]]
Error (desviación + media): 1.294130585811494

SOLUCIONES MACHOS...

Solución tratamiento: 0
[[35.84414624 35.72281252 35.74975154 35.78417619]
 [35.3151425  35.84985546 35.89641246 36.10339303]
 [36.1325738  36.19019357 36.21055507 36.21184921]]
Error (desviación + media): 0.7795516332288077

Solución tratamiento: 1
[[36.23102732 36.24592036 36

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta:

Nuestro algoritmo de fuerza bruta llama a una función que tiene 3 bucles for anidados. Esa función tiene orden 0(n^3).

Teniendo en cuenta que nuestro algoritmo tiene un bucle while, en el peor de los casos donde la mejor solución sea la última, el orden será factorial.

Por tanto, mejor caso O(n^3), peor caso O(n!).

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta:

Voy a implementar un algoritmo genético

### Creación población inicial

In [128]:
# Creo un tratamiento haciendo múltiples permutaciones del grupo inicial
def crear_tratamiento(grupos_iniciales):
  # Me sirve para crear una nueva variable por valor
  tratamiento = copy.deepcopy(grupos_iniciales)

  # Por cada grupo excepto el último porque ya habrá sido utilizado
  for i in range(len(grupos_iniciales)-1):
    # Por cada ternero del grupo
    for j in range(len(grupos_iniciales[i])):
      # Por cada ternero del siguiente grupo
      for k in range(len(grupos_iniciales[i+1])):
        # Intercambio terneros distintos cada vez
        tmp = tratamiento[i+1][k]
        tratamiento[i+1][k] = tratamiento[i][j]
        tratamiento[i][j] = tmp
        
  return tratamiento
  
def crear_poblacion_inicial(num_poblaciones, num_grupos, tamanyo_grupos):
  # Creo una matriz tridimensional vacia para tener todas las poblaciones
  poblacion = np.zeros(shape=(num_poblaciones, num_grupos, tamanyo_grupos))
  
  # Añado la primera población creada aleatoriamente
  poblacion[0] = crear_tratamiento(matriz_grupos_hembras[0])
  
  # El resto de poblaciones las genero en base a la población anterior
  for i in range(1, num_poblaciones):
    poblacion[i] = crear_tratamiento(poblacion[i-1])
  
  return poblacion

tamanyo_grupos = len(terneros_machos.index)//grupos
print("Población inicial generada:\n")
poblacion = crear_poblacion_inicial(4, 3, tamanyo_grupos)
print(poblacion)

Población inicial generada:

[[[35.86294281 35.7773687  35.71552288 35.69453293]
  [35.97306195 35.95690236 35.93814829 35.88926591]
  [35.15138435 35.29687589 35.30369393 35.37521029]]

 [[35.88926591 35.93814829 35.95690236 35.97306195]
  [35.37521029 35.30369393 35.29687589 35.15138435]
  [35.86294281 35.7773687  35.71552288 35.69453293]]

 [[35.15138435 35.29687589 35.30369393 35.37521029]
  [35.69453293 35.71552288 35.7773687  35.86294281]
  [35.88926591 35.93814829 35.95690236 35.97306195]]

 [[35.86294281 35.7773687  35.71552288 35.69453293]
  [35.97306195 35.95690236 35.93814829 35.88926591]
  [35.15138435 35.29687589 35.30369393 35.37521029]]]


### Selección de poblaciones

In [138]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)
  
def calcular_errores(poblacion):
  errores = []
  # Calculamos los errores de la población
  for i in range(len(poblacion)):
    (desv, med) = funcion_objetivo(poblacion[i])
    errores.append(desv + med)
  
  return normalized(errores)

# Devolvemos qué población debemos eliminar
def seleccionar(poblacion):

  errores = calcular_errores(poblacion)
  # Normalizamos los errores
  # Buscamos el índice del elemento que hay que eliminar
  indice_eliminar =  np.argmax(errores)
  
  return indice_eliminar
  
seleccionar(poblacion)

1

### Cruzar

In [175]:
def cruzar(poblacion):
  indice_eliminar = seleccionar(poblacion)

  # Elimino la población con más errores
  errores = calcular_errores(poblacion)
  errores = np.delete(errores, indice_eliminar)
  # Calculo los errores para después asignar probabilidades
  for i in range(len(errores)):
    errores[i] = 1 - errores[i]
  
  # Queremos asignar porcentajes de probabilidad para cruzar
  probabilidades = []
  for i in range(len(errores)):
    probabilidades.append(errores[i]/np.sum(errores))
    
  # Necesito los indices para seleccionar individuos a cruzarse
  indices = [i for i in range(3)]
  eleccion_a = np.random.choice(indices, p=probabilidades)
  eleccion_b = np.random.choice(indices, p=probabilidades)
  # No queremos duplicados
  while (eleccion_a == eleccion_b):
    eleccion_b = np.random.choice(indices, p=probabilidades)
  # Ahora cruzamos
  
cruzar(poblacion)

2
1


(*)Calcula la complejidad del algoritmo 

Respuesta

**Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios**

Respuesta:

En la sección "Creación de la estructura de datos" podrá encontrar una generación aleatoria de terneros machos y hembra.

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta